# Wine Quality Classification with LightGBM

This script trains a LightGBM classifier to predict wine quality categories.

**Dataset**: Wine Quality Dataset
- 1,599 red wine samples
- Target: Quality score (3-8) converted to binary (good/bad)

**Features**:
The dataset includes 13 physicochemical properties:
- `alcohol`: Alcohol content
- `malic_acid`: Malic acid content
- `ash`: Ash content
- `alcalinity_of_ash`: Alcalinity of ash
- `magnesium`: Magnesium content
- `total_phenols`: Total phenols
- `flavanoids`: Flavanoids content
- `nonflavanoid_phenols`: Non-flavanoid phenols
- `proanthocyanins`: Proanthocyanins content
- `color_intensity`: Color intensity
- `hue`: Hue
- `od280/od315_of_diluted_wines`: OD280/OD315 ratio
- `proline`: Proline content

In [ ]:
%pip install lightgbm pandas numpy scikit-learn mlflow

In [ ]:
import os
import argparse
import json
import tempfile
import numpy as np
import pandas as pd
from datetime import datetime

# LightGBM imports
import lightgbm as lgb

# MLflow imports
import mlflow
import mlflow.lightgbm
from mlflow import set_tracking_uri, set_experiment
from mlflow.client import MlflowClient
from mlflow.models import infer_signature

# Scikit-learn imports
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix


def setup_mlflow(mlflow_uri: str, username: str, password: str) -> MlflowClient:
    """Configure MLflow tracking and return client."""
    os.environ["MLFLOW_TRACKING_USERNAME"] = username
    os.environ["MLFLOW_TRACKING_PASSWORD"] = password
    
    set_tracking_uri(mlflow_uri)
    client = MlflowClient(mlflow_uri)
    
    print(f"MLflow tracking URI: {mlflow_uri}")
    return client


def load_and_prepare_data():
    """Load Wine dataset and prepare train/test splits."""
    print("\n" + "=" * 80)
    print("LOADING DATASET")
    print("=" * 80)
    
    # Load dataset
    data = load_wine(as_frame=True)
    X = data.data
    y = data.target
    
    print(f"Dataset: Wine")
    print(f"Samples: {X.shape[0]:,}")
    print(f"Features: {X.shape[1]}")
    print(f"\nFeature names:")
    for i, col in enumerate(X.columns, 1):
        print(f"  {i}. {col}")
    
    print(f"\nTarget distribution:")
    for class_idx in range(3):
        count = (y == class_idx).sum()
        print(f"  Class {class_idx}: {count} samples")
    
    # Split the data
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42, shuffle=True, stratify=y
    )
    
    print(f"\nTrain samples: {X_train.shape[0]:,}")
    print(f"Test samples: {X_test.shape[0]:,}")
    
    return X_train, X_test, y_train, y_test, X.columns.tolist()


def train_model(X_train, y_train, X_test, y_test, hyperparams: dict):
    """Train LightGBM model and return predictions."""
    print("\n" + "=" * 80)
    print("TRAINING MODEL")
    print("=" * 80)
    
    print("Hyperparameters:")
    for key, value in hyperparams.items():
        print(f"  {key}: {value}")
    
    # Create LightGBM datasets
    train_data = lgb.Dataset(X_train, label=y_train)
    test_data = lgb.Dataset(X_test, label=y_test, reference=train_data)
    
    # Train model
    model = lgb.train(
        params=hyperparams,
        train_set=train_data,
        num_boost_round=hyperparams.pop('num_boost_round', 100),
        valid_sets=[train_data, test_data],
        valid_names=['train', 'test'],
        callbacks=[
            lgb.early_stopping(stopping_rounds=20),
            lgb.log_evaluation(period=10)
        ]
    )
    
    print("Training completed!")
    
    # Predictions
    y_train_pred = np.argmax(model.predict(X_train), axis=1)
    y_test_pred = np.argmax(model.predict(X_test), axis=1)
    
    return model, y_train_pred, y_test_pred


def calculate_metrics(y_true, y_pred, dataset_name="Test"):
    """Calculate and return evaluation metrics."""
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average='weighted')
    recall = recall_score(y_true, y_pred, average='weighted')
    f1 = f1_score(y_true, y_pred, average='weighted')
    
    return {
        f"{dataset_name.lower()}_accuracy": accuracy,
        f"{dataset_name.lower()}_precision": precision,
        f"{dataset_name.lower()}_recall": recall,
        f"{dataset_name.lower()}_f1": f1
    }


def log_to_mlflow(model, X_train, y_train, X_test, y_test, 
                  y_train_pred, y_test_pred, hyperparams, feature_names):
    """Log model, parameters, and metrics to MLflow."""
    print("\n" + "=" * 80)
    print("LOGGING TO MLFLOW")
    print("=" * 80)
    
    # Log hyperparameters
    for key, value in hyperparams.items():
        mlflow.log_param(key, value)
    
    # Calculate and log metrics
    train_metrics = calculate_metrics(y_train, y_train_pred, "Train")
    test_metrics = calculate_metrics(y_test, y_test_pred, "Test")
    all_metrics = {**train_metrics, **test_metrics}
    
    for metric_name, metric_value in all_metrics.items():
        mlflow.log_metric(metric_name, metric_value)
    
    print("\nModel Performance:")
    print(f"  Training Accuracy: {train_metrics['train_accuracy']:.4f}")
    print(f"  Training F1: {train_metrics['train_f1']:.4f}")
    print(f"  Test Accuracy: {test_metrics['test_accuracy']:.4f}")
    print(f"  Test Precision: {test_metrics['test_precision']:.4f}")
    print(f"  Test Recall: {test_metrics['test_recall']:.4f}")
    print(f"  Test F1: {test_metrics['test_f1']:.4f}")
    
    # Print confusion matrix
    cm = confusion_matrix(y_test, y_test_pred)
    print(f"\n  Confusion Matrix:")
    print(f"  {cm}")
    
    # Create output with explicit column name
    predictions = model.predict(X_train[:1])  # Keep probabilities
    y_pred_df = pd.DataFrame(predictions, columns=["class_0_prob", "class_1_prob", "class_2_prob"])
    
    # Create model signature
    signature = infer_signature(X_train, y_pred_df)
    input_example = X_test.head(1)
    
    # Save and log model
    with tempfile.TemporaryDirectory() as tmpdir:
        local_model_path = os.path.join(tmpdir, "model")
        
        mlflow.lightgbm.save_model(
            model,
            local_model_path,
            signature=signature,
            input_example=input_example
        )
        
        mlflow.log_artifacts(local_model_path, artifact_path="model")
        print("Model artifacts logged successfully!")
    
    return all_metrics


def create_sample_payload(X_test, y_test, model, feature_names):
    """Create realistic sample prediction payload."""
    # Get a sample
    sample_idx = 0
    sample = X_test.iloc[sample_idx]
    actual_class = y_test.iloc[sample_idx]
    
    # Predict
    predicted_class = np.argmax(model.predict(sample.values.reshape(1, -1)), axis=1)[0]
    
    return {
        "features": sample.to_dict(),
        "actual_class": int(actual_class),
        "predicted_class": int(predicted_class)
    }


def register_model(client: MlflowClient, model_name: str, run_id: str, experiment_id: str):
    """Register model in MLflow Model Registry."""
    print("\n" + "=" * 80)
    print("REGISTERING MODEL")
    print("=" * 80)
    
    model_uri = f"runs:/{run_id}/model"
    
    # Create registered model if it doesn't exist
    try:
        client.get_registered_model(model_name)
        print(f"Model '{model_name}' already exists in registry")
    except Exception:
        try:
            client.create_registered_model(model_name)
            print(f"Created registered model: {model_name}")
        except Exception as e:
            print(f"Could not create registered model: {e}")
    
    # Create model version
    try:
        result = client.create_model_version(
            name=model_name,
            source=model_uri,
            run_id=run_id
        )
        print(f"Model version registered successfully!")
        print(f"   Model Name: {model_name}")
        print(f"   Version: {result.version}")
        print(f"   Run ID: {run_id}")
        return result.version
    except Exception as e:
        print(f"Model registration failed (model still usable via run URI): {e}")
        print(f"   You can deploy using: mlflow-artifacts:/{experiment_id}/{run_id}/artifacts/model")
        return None


def print_deployment_info(run_id: str, experiment_id: str, sample_payload: dict):
    """Print deployment instructions and sample payloads."""
    print("\n" + "=" * 80)
    print("TRAINING COMPLETE!")
    print("=" * 80)
    
    print(f"\nRun Information:")
    print(f"  Run ID: {run_id}")
    print(f"  Experiment ID: {experiment_id}")
    print(f"  Model URI: mlflow-artifacts:/{experiment_id}/{run_id}/artifacts/model")
    
    print("\n" + "=" * 80)
    print("DEPLOYMENT PAYLOAD (deploy-model API)")
    print("=" * 80)
    
    deploy_payload = {
        "serve_name": "wine-lightgbm-classifier",
        "model_uri": f"mlflow-artifacts:/{experiment_id}/{run_id}/artifacts/model",
        "env": "local",
        "cores": 2,
        "memory": 4,
        "node_capacity": "spot",
        "min_replicas": 1,
        "max_replicas": 3
    }
    
    print(json.dumps(deploy_payload, indent=2))
    
    print("\n" + "=" * 80)
    print("SAMPLE PREDICTION PAYLOAD")
    print("=" * 80)
    
    predict_payload = {
        "features": sample_payload["features"]
    }
    
    print(json.dumps(predict_payload, indent=2))
    
    print(f"\nExpected Output:")
    print(f"  Actual Class: {sample_payload['actual_class']}")
    print(f"  Model Prediction: {sample_payload['predicted_class']}")
    
    print("\n" + "=" * 80)
    print("FEATURE DESCRIPTIONS")
    print("=" * 80)
    features = sample_payload["features"]
    print(f"""
  alcohol:                  {features['alcohol']:.2f}
  malic_acid:               {features['malic_acid']:.2f}
  ash:                      {features['ash']:.2f}
  alcalinity_of_ash:        {features['alcalinity_of_ash']:.2f}
  magnesium:                {features['magnesium']:.2f}
  total_phenols:            {features['total_phenols']:.2f}
  flavanoids:               {features['flavanoids']:.2f}
  nonflavanoid_phenols:     {features['nonflavanoid_phenols']:.2f}
  proanthocyanins:          {features['proanthocyanins']:.2f}
  color_intensity:          {features['color_intensity']:.2f}
  hue:                      {features['hue']:.2f}
  od280/od315_of_diluted_wines: {features['od280/od315_of_diluted_wines']:.2f}
  proline:                  {features['proline']:.2f}
    """)


In [ ]:
def main():
    parser = argparse.ArgumentParser(description="Train LightGBM Wine Classification Model")
    parser.add_argument(
        "--mlflow-uri",
        default="http://darwin-mlflow-lib.darwin.svc.cluster.local:8080",
        help="MLflow tracking URI"
    )
    parser.add_argument(
        "--username",
        default="abc@gmail.com",
        help="MLflow username"
    )
    parser.add_argument(
        "--password",
        default="password",
        help="MLflow password"
    )
    parser.add_argument(
        "--experiment-name",
        default="wine_lightgbm_classification",
        help="MLflow experiment name"
    )
    parser.add_argument(
        "--model-name",
        default="WineLightGBMClassifier",
        help="Registered model name"
    )
    
    args, _ = parser.parse_known_args()
    
    print("\n" + "=" * 80)
    print("WINE QUALITY CLASSIFICATION WITH LIGHTGBM")
    print("=" * 80)
    print(f"Started at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    
    # Setup MLflow
    client = setup_mlflow(args.mlflow_uri, args.username, args.password)
    set_experiment(experiment_name=args.experiment_name)
    print(f"Experiment: {args.experiment_name}")
    
    # Load data
    X_train, X_test, y_train, y_test, feature_names = load_and_prepare_data()
    
    # Define hyperparameters
    hyperparams = {
        "objective": "multiclass",
        "num_class": 3,
        "metric": "multi_logloss",
        "num_leaves": 31,
        "learning_rate": 0.05,
        "feature_fraction": 0.9,
        "bagging_fraction": 0.8,
        "bagging_freq": 5,
        "num_boost_round": 100,
        "verbose": -1
    }
    
    # Start MLflow run
    with mlflow.start_run(run_name=f"lightgbm_wine_{datetime.now().strftime('%Y%m%d_%H%M%S')}"):
        # Train model
        model, y_train_pred, y_test_pred = train_model(
            X_train, y_train, X_test, y_test, hyperparams.copy()
        )
        
        # Log to MLflow
        metrics = log_to_mlflow(
            model, X_train, y_train, X_test, y_test,
            y_train_pred, y_test_pred, hyperparams, feature_names
        )
        
        # Get run information
        run_id = mlflow.active_run().info.run_id
        experiment_id = mlflow.active_run().info.experiment_id
        
        # Create sample payload
        sample_payload = create_sample_payload(X_test, y_test, model, feature_names)
    
    # Register model (outside of run context)
    version = register_model(client, args.model_name, run_id, experiment_id)
    
    # Print deployment information
    print_deployment_info(run_id, experiment_id, sample_payload)
    
    print("\nScript completed successfully!")
    print("=" * 80 + "\n")


if __name__ == "__main__":
    main()
